In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Embedding, LSTM
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import drive


import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50, InceptionV3, VGG16, EfficientNetB0, DenseNet121, MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from google.colab import drive



# Mount Google Drive
drive.mount('/content/drive')

# 1. Set Up Paths
image_folder_path = '/content/drive/My Drive/instabot/ds/images'
csv_path = '/content/drive/My Drive/instabot/ds/datasets.csv'
model_save_path = '/content/drive/My Drive/models1/'
import glob

# Function to load all images and metadata
def load_images_and_metadata(csv_path, image_folder_path):
    data, labels, metadata = [], [], []
    df = pd.read_csv(csv_path)

    # Get all image files in the folder with specified extensions
    all_images = glob.glob(os.path.join(image_folder_path, '*'))
    valid_extensions = ('.jpg', '.jpeg', '.png')
    all_images = [img for img in all_images if img.lower().endswith(valid_extensions)]

    print("Found images:")
    for img in all_images:
        print(img)

    for _, row in df.iterrows():
        # Match the image path using the `image` column
        img_name = row['image']
        img_path = os.path.join(image_folder_path, img_name)
        print(f"Processing image: {img_path}")
        # Ensure the image exists in the folder
        if img_path in all_images:
            img = cv2.imread(img_path)
            if img is not None:
                img_resized = cv2.resize(img, (128, 128))  # Resize to a consistent size
                img_normalized = img_resized / 255.0  # Normalize pixel values
                data.append(img_normalized)
                labels.append(row['category'])
                metadata.append({
                    'objects': row['objects'],
                    'distance': row['distance']
                })
            else:
                print(f"Warning: Unable to read image - {img_path}")
        else:
            print(f"Warning: Image not found in folder - {img_name}")

    return np.array(data), np.array(labels), metadata
# Load images and metadata

X, y_labels, metadata = load_images_and_metadata(csv_path, image_folder_path)

# Encode labels
y_classes = sorted(set(y_labels))
label_to_idx = {label: idx for idx, label in enumerate(y_classes)}
print(y_classes, label_to_idx)
y = np.array([label_to_idx[label] for label in y_labels])
y = to_categorical(y, num_classes=len(y_classes))


# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# TPU Strategy
try:
    import tensorflow as tf
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    strategy = tf.distribute.TPUStrategy(tpu)
    print("Running on TPU")
except ValueError:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU/GPU")




# Define Pre-trained Models
def create_pretrained_model(base_model, num_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    return Model(inputs=base_model.input, outputs=predictions)

# List of Models
model_classes = [
    (EfficientNetB0, 'EfficientNetB0'),
    (DenseNet121, 'DenseNet121'),
    (MobileNet, 'MobileNet')
]
IMG_SIZE = 128
# Train and Save Models
for model_class, model_name in model_classes:
    with strategy.scope():
        print(f"Preparing {model_name}...")
        base_model = model_class(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
        model = create_pretrained_model(base_model, len(y_classes))
        model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(filepath=os.path.join(model_save_path, f"{model_name}.keras"),
                                 save_best_only=True,
                                 monitor='val_accuracy',
                                 mode='max')

    print(f"Training {model_name}...")
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=100, batch_size=32, callbacks=[checkpoint])
    print(f"{model_name} training complete and saved to Google Drive.")




# 3. Define Model Architectures
def build_model_1():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(y_classes), activation='softmax')
    ])
    return model

def build_model_2():
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.3),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(len(y_classes), activation='softmax')
    ])
    return model

def build_model_3():
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.1),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.1),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(len(y_classes), activation='softmax')
    ])
    return model

def build_model_4():
    model = Sequential([
        Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.4),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.4),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(len(y_classes), activation='softmax')
    ])
    return model

def build_model_5():
    model = Sequential([
        Conv2D(32, (5, 5), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        Conv2D(64, (5, 5), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(0.2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(len(y_classes), activation='softmax')
    ])
    return model



def create_resnet(input_shape, num_classes):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

def create_vgg16(input_shape, num_classes):
    base_model = VGG16(weights=None, include_top=False, input_shape=input_shape)
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model



# 4. Train and Save Models
models = [build_model_1, build_model_2, build_model_3, build_model_4, build_model_5]
# model_names = ["model_1.h5", "model_2.h5", "model_3.h5", "model_4.h5", "model_5.h5"]
model_names = ["model_1.keras", "model_2.keras", "model_3.keras", "model_4.keras", "model_5.keras"]
for i, build_model in enumerate(models):
    with strategy.scope():
        model = build_model()
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(filepath=os.path.join(model_save_path, model_names[i]),
                                 save_best_only=True,
                                 monitor='val_accuracy',
                                 mode='max')

    print(f"Training {model_names[i]}...")
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=100, batch_size=6, callbacks=[checkpoint])
    print(f"{model_names[i]} training complete and saved to Google Drive.")


Streaming output truncated to the last 5000 lines.
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with_handlebars_visible)_2.jpg
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with_handlebars_visible)_3.jpg
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with_handlebars_visible)_4.jpg
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with_handlebars_visible)_5.jpg
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with_handlebars_visible)_6.jpg
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with_handlebars_visible)_7.jpg
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with_handlebars_visible)_8.jpg
Processing image: /content/drive/My Drive/instabot/ds/images/Cycling_on_a_city_street_(POV_with